# Conversion of npy to s2p 

This is used to implement and test the feature suggested by reviewer 2 - allowing easier use of track2p by people using pipelines other than suite2p

Inputs:
- rois.npy (bool ndarray of shape: (n_roi, n_px_y, n_px_x))
- fov.npy (float32 ndarray of shape: (n_px_y, n_px_x))            - IMPORTANT: ROIs and FOV must be aligned (this is done by default in suite2p outputs, but might need to be registered manually in other cases)
- traces.npy (float32 ndarray of shape: (n_roi, n_tstamps))

Outputs:
- suite2p folder

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
nplanes = 1
plane = 0

In [ ]:
# these are the paths to the datasets that will be used to 
subject_npy_path = '/Users/jure/Documents/cossart_lab/data/jm/jm039_npy/'
# npy_paths are all the subfolders of subject_path ending in _a
npy_paths = [os.path.join(f.path, 'data_npy', f'plane{plane}') for f in os.scandir(subject_npy_path) if f.is_dir() and f.name.endswith('_a')]
npy_paths.sort()
print(npy_paths)


s2p_paths = []
for npy_path in npy_paths:
    print(npy_path)
    # remove _npy
    s2p_path = npy_path.replace('jm039_npy', 'jm039')
    s2p_path = s2p_path.replace('data_npy', 'suite2p')
    s2p_paths.append(s2p_path)

In [ ]:
path = npy_paths[0]  # Example path, you can loop through npy_paths if needed



In [ ]:
for (npy_path, s2p_path) in zip(npy_paths, s2p_paths):
    print(s2p_path)
    
    F = np.load(os.path.join(npy_path, 'F.npy'))
    fov = np.load(os.path.join(npy_path, 'fov.npy'))
    rois = np.load(os.path.join(npy_path, 'rois.npy'))



    if not os.path.exists(s2p_path):
        os.makedirs(s2p_path)
    else:
        print(f"Directory {s2p_path} already exists, skipping... (Delete or rename it if you want to overwrite)")
        continue
    
    np.save(os.path.join(s2p_path, 'F.npy'), F)

    ops = {'meanImg': fov}
    ops['nchannels'] = 1  # Assuming single channel
    ops['fs'] = 30  # Assuming a sampling frequency of 30 Hz
    ops['nframes'] = F.shape[1]
    np.save(os.path.join(s2p_path, 'ops.npy'), ops)

    # stat is a list of dictionaries, each with keys 'xpix', 'ypix'
    stat = []
    for i in range(rois.shape[0]):
        # TODO: make sure using the correct axis convention!!!
        ypix, xpix = np.where(rois[i] > 0)
        med = [int(np.median(ypix).item()), int(np.median(xpix).item())]
        stat.append({'xpix': xpix, 'ypix': ypix, 'med': med})

    np.save(os.path.join(s2p_path, 'stat.npy'), stat)
            
    # iscell is two columns of 1 the columns are length n_cells
    n_cells = len(stat)
    iscell = np.ones((n_cells, 2), dtype=int)
    np.save(os.path.join(s2p_path, 'iscell.npy'), iscell)
    print(f"Saved to {s2p_path}")


In [ ]:
med = [int(np.median(ypix).item()), int(np.median(xpix).item())]

med